# Hot song recommendations

Build song recommendation out of the k=100 nearest neighbors

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib as plt
import importlib
from pyspark.ml.feature import Tokenizer, CountVectorizer, MinHashLSH
from pyspark.sql.types import IntegerType, StringType, ArrayType

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load and prep data

* Load the full data set
* Load the picked k=100 approx Nearest Neighbor results
* Build song recommdations based on songs in nearest playlist

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

Get the ranked popularity of songs in the mpd.

In [ ]:
cv = CountVectorizer(inputCol="track_uri", outputCol="features", minDF=2, vocabSize=2000000)

In [ ]:
model=cv.fit(mpd_all.select("pid", "tracks.track_uri"))

In [ ]:
result=model.transform(mpd_all.select("pid", "tracks.track_uri"))

In [ ]:
#model, result = mpd.vectorizecol(mpd_all.select("pid", "tracks.track_uri"), "track_uri", "features", 2000000)

In [ ]:
result.printSchema()

In [ ]:
result.count()

In [ ]:
importlib.reload(mpd)

In [ ]:
vdf = mpd.buildvocabdf(spark, model.vocabulary)

In [ ]:
vdf.show(5)

In [ ]:
vdf.describe("tid").show()

In [ ]:
vdf.printSchema()

In [ ]:
vdf.count()

Get the Hot100 playlists that match the challenge set.

In [ ]:
hot100 = spark.createDataFrame(pd.read_pickle("ex-neighborpl.pkl"))

In [ ]:
hot100 = spark.createDataFrame(pd.read_pickle("neighborpl.pkl"))

In [ ]:
hot100.orderBy("pid").show(5)

In [ ]:
arraylength = f.udf(lambda x: len(x), IntegerType())

In [ ]:
h100cnt = hot100.withColumn("reclen", arraylength(hot100.recpl))

In [ ]:
h100cnt.orderBy("reclen").show()

In [ ]:
h100cnt.groupBy("reclen").count().orderBy("reclen").show(5)

In [ ]:
h100cnt.orderBy("reclen").groupBy("reclen").count().describe("count").show()

In [ ]:
h100cnt.describe("reclen").show()

We can see that most results will have gotten 100 neighbors

In [ ]:
mpd.plothist(h100cnt, "reclen", 11)

In [ ]:
h100 = hot100.select("pid", f.explode("recpl").alias("recpid"))

In [ ]:
h100withtracks = h100.join(result, result.pid == h100.recpid).drop(result.pid).drop(result.features).orderBy("pid")

In [ ]:
h100withtracks.show(5)

In [ ]:
h100withtracks.count()

## Elliminate duplicates with subtracting dataframes

It's possible to [use subtraction to remove the duplicates](https://stackoverflow.com/a/42380533)

Don't actually do this here though. Save it till after all the ordering takes place, otherwise we lose duplicates and the ability to count popularity.

challengetracks = recdf.select("pid", f.explode("tracks.track_uri").alias("track"))

challengetracks.count()

challengetracks.where(challengetracks.pid == 1000000).count()

tracklist = h100withtracks.select("pid", f.explode("track_uri").alias("track"))

tracklist.count()

newtracks = tracklist.subtract(challengetracks)

newtracks.count()

## Get the ranked resutls of tracks from the recommended neighboring playlists.

Make sure the elliminated tracks don't loose detail for a playlist.  The pid 1000000 order by count is different when the tracks aren't elliminated.   However the total count of tracks matches the expected difference.  646 original, less 5 given is 641.

In [ ]:
trackrank = h100withtracks.select("pid", f.explode("track_uri").alias("track")).groupBy("pid","track").count().sort(f.desc("count"))

In [ ]:
trackrank.orderBy("pid", f.desc("count")).show(5)

In [ ]:
trackrank.where(trackrank.pid == 1000000).count()

Commented out for now favoring ellimination at the end

trackrank = newtracks.groupBy("pid","track").count().sort(f.desc("count"))

trackrank.orderBy("pid", f.desc("count")).show(5)

trackrank.where(trackrank.pid == 1000000).count()

In [ ]:
trackrank.where(trackrank.pid == 1000000).orderBy(f.asc("count")).show()

In [ ]:
trackrank.describe("count").show()

In [ ]:
trackrank.printSchema()

### Exlore a single playlist

In [ ]:
testpid = 1000061

In [ ]:
trackrank.where(f.col("pid") == testpid).show()

In [ ]:
trackrank.where(f.col("pid") == testpid).count()

Add the global rank

In [ ]:
grank=trackrank.join(vdf, trackrank.track == vdf.term).drop(vdf.term)

In [ ]:
grank.printSchema()

Here is the track recommendation for one playlist based on the popularity of the track in the neighborhood with additional sorting by the globab popularity. Global popularity is based on count vecorizer with most popular recieving the lowest value.

In [ ]:
grank.where(f.col("pid") == testpid).orderBy(f.desc("count"), f.asc("tid")).show()

## Eliminate tracks included in the search

In [ ]:
mpd_test=spark.read.json("../mpd-challenge/challenge_set.json", multiLine=True)

In [ ]:
cpl=mpd_test.select(f.explode("playlists").alias("playlist"))

In [ ]:
recdf=cpl.select("playlist.pid", "playlist.tracks")

In [ ]:
recdf.describe("pid").show()

test a playlist

In [ ]:
existingtracks = recdf.where(recdf.pid == testpid).select(f.explode("tracks.track_uri").alias("track"))

In [ ]:
existingtracks.printSchema()

In [ ]:
existingtracks.show()

In [ ]:
existingtracks.toPandas()["track"].tolist()

In [ ]:
trank = grank.where(f.col("pid") == testpid).where(~grank.track.isin(existingtracks.toPandas()["track"].tolist()))

In [ ]:
trank.orderBy(f.desc("count"), f.asc("tid")).show()

## Iterate over search results and provide track list

In [ ]:
def gettracks(chpl, grank, recdf):
    # get the challenge playlist id
    testpid = chpl.pid
    
    # get the provided tracks
    existingtracks = recdf.where(recdf.pid == testpid).select(f.explode("tracks.track_uri").alias("track"))
    
    # get the tracks from the global rank
    df = grank.where(f.col("pid") == testpid).where(~grank.track.isin(existingtracks.toPandas()["track"].tolist()))
    
    tracklist = df.orderBy(f.desc("count"), f.asc("tid")).toPandas()["track"].tolist()
    
    recommend = { "pid": testpid, "tracks": [tracklist]}
    #print("DEBUG: " + testpid + " " + pidlist)
    
    return recommend


In [ ]:
recommended = pd.DataFrame({"pid":0, "tracks":[]})

Don't do this. It's awfully slow.

for row in hot100.limit(10).rdd.collect():
    rec = gettracks(row, grank, recdf)
    recommended = recommended.append(pd.DataFrame(rec))

In [ ]:
recommended

In [ ]:
recommended["tracks"].apply(lambda x: len(x))

recommended.to_pickle("rectracks.pkl")

recommended.to_csv("rectracks.csv")

## The iterrate-over-data-set is too damn slow 

The global ranking doesn't take much time even with a few cores.

In [ ]:
grank.orderBy("pid", f.desc("count"), f.asc("tid")).show()

Add monotonic id to preserve order.

In [ ]:
tgrank = grank.withColumn("mid", f.monotonically_increasing_id())

In [ ]:
tgrank = grank.orderBy("pid", f.desc("count"), f.asc("tid")).withColumn("mid", f.monotonically_increasing_id())

Now we have an mid that preserves the global order of our data set and can be used to sort tracks within playlists independent of nearest neighbor or global popularity counts.

In [ ]:
tgrank.orderBy("mid").show()

tgrank = tgrank.orderBy("pid", f.desc("count"), f.asc("tid")).withColumn("mid", f.monotonically_increasing_id()).drop("count").drop("tid")

In [ ]:
tgrank.count()

In [ ]:
tgrank = tgrank.drop("count").drop("tid")

In [ ]:
tgrank.printSchema()

### Elliminate duplicates with subtracting dataframes

In [ ]:
challengetracks = recdf.select("pid", f.explode("tracks.track_uri").alias("track"))

In [ ]:
challengetracks.printSchema()

Impart monotonic id on challenge set to create identical rows

In [ ]:
ctracks = tgrank.join(challengetracks, [challengetracks.pid == tgrank.pid, challengetracks.track == tgrank.track]).drop(challengetracks.pid).drop(challengetracks.track)

In [ ]:
ctracks.printSchema()

In [ ]:
ctracks.orderBy("pid", "mid").show(truncate=False)

In [ ]:
newtracks = tgrank.subtract(ctracks)

Comparing to the above results it's clear we've ellimnated the provided tracks

In [ ]:
newtracks.count()

In [ ]:
newtracks.orderBy("pid", "mid").show()

Need to do the ellimination sooner because this approach loses the desired sorting.

Actually can do it here if we introduce a row index and do a join with the challenge set to get the duplicates with the same row id.  Then we can subtract and have identical rows and also retain the relative ordering.

Monotonically increasing should be fine since all we care about is sorting.

### Save data 

In [ ]:
trackrec = newtracks.orderBy("pid", "mid").groupBy("pid").agg(f.collect_list("track").alias("tracks"))

In [ ]:
trackrec.printSchema()

In [ ]:
trackrec.show(5)

In [ ]:
trackrec = trackrec.withColumn("len", arraylength(trackrec.tracks))

In [ ]:
trackrec.where(trackrec.len < 500).show()

In [ ]:
trackrec.count()

In [ ]:
trackrec.where(trackrec.len < 500).count()

In [ ]:
trunc500 = f.udf(lambda x: x[0:500], ArrayType(StringType()))

In [ ]:
cleanlist = trackrec.withColumn("first500", trunc500(trackrec.tracks))

## Format output

In [ ]:
trpd = cleanlist["pid","first500"].toPandas()

In [ ]:
trpd.to_csv("reclist.csv")

In [ ]:
trpd.to_csv("reclist2.csv", quoting=0)

In [ ]:
trpd["formatted"] = trpd["pid"].map(str) + ", " + trpd["first500"].apply(', '.join)

In [ ]:
trpd["formatted"]

In [ ]:
type(trpd)

In [ ]:
trpd["formatted"].to_csv("recpl.csv", index=False, header=False)

In [ ]:
!echo -e "team_info,main,jprorama,jpr@uab.edu\n\n" > recpl-clean.csv

Fix the output becuase the quoting flag doesn't work above

In [ ]:
!sed -e 's/^"//' -e 's/"$//' recpl.csv >> recpl-clean.csv